참고 링크1 : https://ai.google.dev/gemini-api/docs/models?hl=ko <br>
참고 링크2 : https://googleapis.github.io/python-genai/

In [ ]:
!pip install google-genai python-dotenv

In [6]:
#!pip install google-genai

True

### 텍스트 생성

In [9]:
import os

api_key = os.getenv("GOOGLE_API_KEY")

**genai.Client** : https://googleapis.github.io/python-genai/genai.html#module-genai.client

In [4]:
from google import genai

# import google.generativeai as genai

# 클라이언트 인스턴스 생성
client = genai.Client(api_key=api_key)

# 모델을 지정하고 컨텐츠의 답변을 요청
response = client.models.generate_content(
    # vertexai=False,
    model="gemini-2.5-flash",
    contents="대한민국의 수도는 어디인가요?",
)
print(response.text)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


대한민국의 수도는 **서울**입니다.


In [34]:
response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text='대한민국의 수도는 **서울**입니다.'
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>,
      index=0
    ),
  ],
  model_version='gemini-2.5-flash',
  response_id='oVWZaM2oLJS7qtsPt6TFwQs',
  sdk_http_response=HttpResponse(
    headers=<dict len=11>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=10,
    prompt_token_count=10,
    prompt_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=10
      ),
    ],
    thoughts_token_count=27,
    total_token_count=47
  )
)

In [39]:
# response.candidates[0].content.parts[0].text
response.candidates[0].content.parts[0].text

'대한민국의 수도는 **서울**입니다.'

In [46]:
response.usage_metadata

ModalityTokenCount(
  modality=<MediaModality.TEXT: 'TEXT'>,
  token_count=10
)

In [14]:
# 토큰 사용량 정보 출력

# query 토큰 수
print(response.usage_metadata.prompt_token_count)

# throughts 토큰 수(생각하는 토큰 수)
print(response.usage_metadata.thoughts_token_count)

# query 토근수 + throughts 토큰 수 + 응답 토근 수
print(response.usage_metadata.total_token_count)

10
27
47


### 시스템 안내 및 기타 구성

In [ ]:
from google import genai
from google.genai import types

client = genai.Client()

# Google Gemini 모델과 직접 상호작용하여 콘텐츠를 생성하는 데 사용되는 핵심 함수
response = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        # 시스템 인스트럭션
        system_instruction="당신은 고양이이고, 이름은 야옹이입니다."
    ),
    contents="안녕",
)

print(response.text)

야옹! 😼


### GenerateContentConfig : temperature 설정

In [53]:
from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=["AI가 어떻게 동작하는지 3문장으로 설명줘."],
    config=types.GenerateContentConfig(temperature=0.1),
)
print(response.text)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


AI는 방대한 데이터를 분석하여 숨겨진 패턴과 규칙을 학습합니다. 이 학습된 지식을 바탕으로 새로운 상황을 이해하고 예측하거나 문제를 해결합니다. 마치 사람처럼 경험을 통해 똑똑해지며, 주어진 목표를 달성하도록 설계됩니다.


### 멀티모달 입력

In [21]:
# from PIL import Image
# from google import genai

# client = genai.Client()

# image = Image.open("organ.jpg")
# response = client.models.generate_content(
#     model="gemini-2.5-flash",
#     contents=[image, "Tell me about this instrument"]
# )
# print(response.text)

위의 코드는 에러가 발생된다. 다음 페이지의 코드를 참조하기 바람 <br>
https://ai.google.dev/gemini-api/docs/image-understanding?hl=ko

In [57]:
import os

# 현재 작업 디렉토리 경로를 가져옵니다.
HOME = os.getcwd()
HOME

'c:\\Users\\park0\\github\\without-langchain\\tutorials'

In [59]:
from google.genai import types

# 이미지 파일을 읽어와서 바이너리 데이터로 변환
with open("../data/organ.jpg", "rb") as f:
    image_bytes = f.read()


response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        # 바이너리 타입의 이미지
        types.Part.from_bytes(
            # 바이너리로 읽은 데이터를 첨부하기
            data=image_bytes,
            mime_type="image/jpeg",
        ),
        # text 타입의 프롬프트
        "이 악기에 대해 3문장으로 설명해줘",
    ],
)

print("=== 응답 ===")
print(response.text)
print("\n=== 토큰 사용량 ===")
print(f"입력 토큰: {response.usage_metadata.prompt_token_count}")
print(f"생각 토큰: {response.usage_metadata.thoughts_token_count}")
print(f"출력 토큰: {response.usage_metadata.candidates_token_count}")
print(f"총 토큰: {response.usage_metadata.total_token_count}")

=== 응답 ===
이 사진은 여러 단의 건반(매뉴얼), 페달 건반, 그리고 수많은 스톱(Stop) 손잡이가 있는 웅장하고 복잡한 파이프 오르간을 보여줍니다. 연주자는 이 스톱들을 조작하여 다양한 음색과 음량을 조절하며 풍부한 소리를 만들어냅니다. 특히 섬세하게 조각된 목재와 정교한 제어 장치들은 이 악기가 가진 뛰어난 장인정신과 방대한 음향적 가능성을 잘 보여줍니다.

=== 토큰 사용량 ===
입력 토큰: 271
생각 토큰: 1114
출력 토큰: 115
총 토큰: 1500


In [ ]:
from PIL import Image
from google import genai
from google.genai import types
import io

client = genai.Client()

image = Image.open("../data/organ.jpg")

# Convert image to bytes
img_byte_arr = io.BytesIO()
image.save(img_byte_arr, format="JPEG")
img_byte_arr = img_byte_arr.getvalue()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        types.Content(
            role="user",
            parts=[
                types.Part(text="이 악기에 대해 설명해줘"),
                types.Part(
                    inline_data=types.Blob(mime_type="image/jpeg", data=img_byte_arr)
                ),
            ],
        )
    ],
)
print(response.text)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


이 악기는 **오르간(Organ)**입니다. 특히, 파이프 오르간의 연주자용 콘솔(Console) 부분을 보여줍니다. 오르간은 건반 악기이자 동시에 관악기의 원리를 사용하는 독특한 악기입니다. '악기의 제왕'이라 불릴 만큼 웅장하고 풍부한 소리를 자랑합니다.

**주요 특징 및 구성 요소:**

1.  **수동 건반 (Manuals):**
    *   사진 중앙에 여러 단(일반적으로 2단에서 5단 이상)으로 쌓여 있는 것이 손으로 연주하는 건반입니다. 각 건반은 특정 음색 그룹(오르간 악보에서는 '디비전'이라고도 함)을 담당합니다.
    *   사진에는 두 단의 수동 건반이 명확히 보입니다.

2.  **페달 건반 (Pedalboard):**
    *   하단에 발로 연주하는 거대한 건반입니다. 주로 저음부를 담당하며, 오르간 연주의 특징 중 하나인 손과 발의 독립적인 사용을 가능하게 합니다.

3.  **스톱 (Stops):**
    *   건반 양쪽에 있는 수많은 버튼이나 손잡이들이 '스톱'입니다. 이 스톱을 조작하여 파이프 오르간의 다양한 음색(예: 플루트 소리, 트럼펫 소리, 현악기 소리 등)을 선택하고 조합할 수 있습니다. 각 스톱은 특정 음고(Pitch, 예: 8', 4', 2' 등)를 가진 파이프 세트를 활성화시킵니다.

4.  **표현 페달 (Expression Pedals / Swell Pedals):**
    *   페달 건반 위쪽 중앙에 발로 조작하는 페달들이 보입니다. 이 페달들은 소리의 크기(음량)를 조절하는 역할을 합니다.

**작동 원리 (파이프 오르간 기준):**
오르간은 건반과 스톱을 조작하면 파이프를 통해 공기가 흘러나와 소리를 냅니다. 수백 개에서 수만 개에 이르는 다양한 크기, 모양, 재질의 파이프들이 각각 다른 음색과 음높이를 만들어냅니다. 스톱은 이 파이프들 중 어떤 그룹을 사용할지 선택하는 역할을 합니다.

**소리의 특징 및 용도:**
오르간은 작은 속삭임 같은 소리부터 건물 전체를 울리는 웅장하고 강력한 소

### 스트리밍 응답

더 원활한 상호작용을 위해 스트리밍을 사용하여 GenerateContentResponse 인스턴스를 생성되는 대로 **점진적으로 수신**하세요.

In [62]:
from google import genai

client = genai.Client()

response = client.models.generate_content_stream(
    model="gemini-2.5-flash", contents=["인공지능에 대해 10문장으로 설명해줘"]
)
for chunk in response:
    print(chunk.text, end="")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


인공지능(AI)은 컴퓨터 시스템이 인간의 지능을 모방하도록 만드는 기술입니다.
이는 학습, 문제 해결, 의사 결정, 그리고 언어 이해와 같은 능력을 포함합니다.
AI는 주로 방대한 데이터를 분석하고 패턴을 학습하여 스스로 성능을 향상시킵니다.
이러한 학습 과정을 통해 특정 작업을 수행하거나 예측을 할 수 있게 됩니다.
크게 머신러닝, 딥러닝 등의 세부 분야로 나뉘며, 각기 다른 방식으로 지능을 구현합니다.
현재 우리가 접하는 대부분의 AI는 특정 목적에 특화된 '약한 인공지능'입니다.
예를 들어, 음성 인식 비서, 추천 시스템, 자율주행차가 대표적인 AI 활용 사례입니다.
AI는 반복적인 작업을 자동화하고 복잡한 문제를 해결하며, 효율성을 크게 높이는 데 기여합니다.
미래에는 의료, 교육, 환경 등 다양한 분야에서 혁신적인 변화를 가져올 잠재력을 가지고 있습니다.
하지만 윤리적 문제, 데이터 편향, 일자리 변화 등 사회적 책임과 신중한 접근이 필요합니다.

### 멀티턴 대화 (채팅)

In [63]:
from google import genai

client = genai.Client()
chat = client.chats.create(model="gemini-2.5-flash")

response = chat.send_message("나는 우리 집에 2마리 강아지가 있다.")
print(response.text)

response = chat.send_message(
    "그렇다면 우리집에 있는 동물의 발의 수는 어떻게 되는지 간단히 답해봐?"
)
print(response.text)

for message in chat.get_history():
    print(f"role - {message.role}", end=": ")
    print(message.parts[0].text)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


와, 그러시군요! 강아지 두 마리와 함께 지내시면 정말 행복하시겠어요! 🐶🐶

어떤 종류의 강아지들인가요? 이름은 무엇인지도 궁금하네요! 😊
네, 그렇다면 우리 집에 있는 동물의 발은 총 **8개**입니다!

(강아지 2마리 x 발 4개 = 8개) 😊
role - user: 나는 우리 집에 2마리 강아지가 있다.
role - model: 와, 그러시군요! 강아지 두 마리와 함께 지내시면 정말 행복하시겠어요! 🐶🐶

어떤 종류의 강아지들인가요? 이름은 무엇인지도 궁금하네요! 😊
role - user: 그렇다면 우리집에 있는 동물의 발의 수는 어떻게 되는지 간단히 답해봐?
role - model: 네, 그렇다면 우리 집에 있는 동물의 발은 총 **8개**입니다!

(강아지 2마리 x 발 4개 = 8개) 😊


In [32]:
chat.get_history()

[UserContent(
   parts=[
     Part(
       text='나는 우리 집에 2마리 강아지가 있다.'
     ),
   ],
   role='user'
 ),
 Content(
   parts=[
     Part(
       text="""두 마리의 사랑스러운 강아지와 함께하고 계시는군요! 🐶🐶 정말 든든하고 행복하시겠어요.
 
 혹시 강아지들 이름이나 품종은 어떻게 되나요? 궁금하네요!"""
     ),
   ],
   role='model'
 ),
 UserContent(
   parts=[
     Part(
       text='그렇다면 우리집에 있는 동물의 발의 수는 어떻게 되는지 간단히 답해봐?'
     ),
   ],
   role='user'
 ),
 Content(
   parts=[
     Part(
       text="""우리 집에 있는 강아지 두 마리의 발 수는 총 **8개**입니다. 🐾
 
 강아지 한 마리당 발이 4개씩 있으니까요!"""
     ),
   ],
   role='model'
 )]